# Introduction

### Star Formation Rate Comparison Program (SFRCP)

This is the program used for my study, "Comparison of Star Formation Rate in Spiral versus Elliptical Galaxies."

This program will be split into different sections for simplicity, with comments explaining each section of code; this is done so that any other person looking at this code understand it fully.

# IMPORTANT NOTE!

For the program to work, each `.fits` file needs to follow a specific naming scheme.

**Check out [the wiki](https://github.com/Wo0fle/SFRCP/wiki) for more information!**

# Imports and Setup

In [1]:
import csv # to import/export .csv files (comma seperated values)
import numpy as np # to do cool math stuff like means and sums of lists
import matplotlib.pyplot as plt # to plot stuff in graphs and to create images of galaxies
from astropy.io import fits # to read/import all the stuff in the .fits files
from astropy.wcs import WCS # to understand the World Coordinate System (WCS)

prefix = 'ORIGINAL GALAXY FITS/' # the folder containing every galaxies' .fits files
suffix = '.fits' # file extension for the files containing the data about each galaxy
w1 = 'w1_' # in front of every .fits file that is WISE 1 data -- denotes WISE 1 data
w3 = 'w3_' # denotes WISE 3 data
nuv = 'NUV_' # denotes NUV data
fuv = 'FUV_' # denotes NUV data
spiral = 'Spiral' # compared against GALAXY INFO.csv to check galaxy type
elliptical = 'Elliptical' # same as above

# note to self: add w2 and w4

# Galaxy List and Info

In [2]:
origGalaxiesList = ['NGC 300', 'M 99', 'NGC 3938', 'NGC 3184', 'M 100', 'M 74', 'NGC 1304', 'NGC 3265', 'NGC 855', 'NGC 1404', 'NGC 584', 'NGC 5866']
# the list of every galaxies' name that has a .fits file, exactly as it appears in the .fits file (common name, not PGC identifier)
galaxiesList_fits = [sub + suffix for sub in origGalaxiesList]
# adds '.fits' to the end of every galaxies' name

with open('GALAXY INFO.csv') as csvfile:
    galaxyInfo = list(csv.reader(csvfile))
# convert GALAXY INFO.csv into a list
# referenced later to retrieve galaxy name, radius, and type (elliptical or spiral)

# WISE Data Analysis

## DONT USE DATA IT IS ALL MESSSED UP
### Problems:
- Galaxy radii are all messed up
    - Different units? Different scale? Unsure

In [3]:
w1GalaxiesList = [w1 + sub for sub in galaxiesList_fits]
# add 'w1_' to the front of each galaxy name (that already has .fits at the end of each)
w3GalaxiesList = [w3 + sub for sub in galaxiesList_fits]
# add 'w3_' to front

wiseGalaxiesList = w1GalaxiesList + w3GalaxiesList
# combine the two lists from above into one large list containing each file name (for WISE analysis)
# referenced later to create the variable galaxyName

galaxiesListForData = [prefix + sub for sub in wiseGalaxiesList]
# add 'ORIGINAL GALAXY FITS/' to front of each term in wiseGalaxiesList
# used so that SFRCP can actually find/read the files

In [4]:
for loop in range(len(wiseGalaxiesList)): # repeat the program for each term in wiseGalaxiesList
    end = len(wiseGalaxiesList[loop]) - 5
    # used to create galaxyName by removing the '.fits' at the end of each term in wiseGalaxiesList
    galaxyName = wiseGalaxiesList[loop][3:end]
    # each galaxies' name (wow!)
    # removed the 'w1_'/'w3_' at the start end '.fits' at the end of each galaxies' file name
    
    for check in range(13): # there are 13 rows in GALAXY INFO.csv, go through each row
        if galaxyName == galaxyInfo[check][0]: # if galaxyName is the same as the galaxies' name in GALAXY INFO.csv, execute the program
            ########################################################
            galaxyRadius =  float(galaxyInfo[check][2]) / 3600 # retrieve galaxies' radius from GALAXY INFO.csv, then convert to float value so it can be used for math
            ########################################################
            
            # idk whats goin on here but the units are off in some way
            
            galaxyType =  galaxyInfo[check][3] # check GALAXY INFO.csv to see if the current galaxy is spiral or elliptical
            galaxyID = wiseGalaxiesList[loop][0:3] # check GALAXY INFO.csv to see if the current galaxy file is WISE 1 or WISE 3 data
            
            data = fits.getdata(galaxiesListForData[loop])
            header = fits.getheader(galaxiesListForData[loop])
            w = WCS(header)
            rows,cols = np.indices(data.shape)
            ra,dec = w.all_pix2world(cols,rows,1)
            ra_ctr = header['CRVAL1']
            dec_ctr = header['CRVAL2']

            distArray = np.sqrt((ra_ctr - ra)**2 + (dec_ctr - dec)**2)
            # based off the Pythagorean theorem: a**2 + b**2 = c**2
            # creates an array of every distance in the image using the ra/dec of the center point of the image and the ra/dec of other points in the image
            radiusArray = np.zeros(50) # creates an empty array of 50 values to be filled in later
            
            for i in range (len(radiusArray)):
                #radius = (i + 1) * distArray.max() / 50
                radius = (i + 1) * galaxyRadius / 50
                radiusArray[i] = radius
                
            binsMean = np.zeros(50)
            binsSum = np.zeros(50)

            for i in range(len(binsMean)):
                #radius = (i + 1) * distArray.max() / 50
                radius = (i + 1) * galaxyRadius / 50
                radiusInds = np.where(distArray < radius)
                imVals = data[radiusInds]
                radiusMean = np.mean(imVals)
                            
                if (galaxyID == w1):
                    ##################
                    stellarMass = 3.3e2 * radiusMean
                    binsMean[i] = stellarMass
                    ##################
                    
                    #wrong formula -- (Section 3.2 -- WISE1 to Stellar Mass)
                    #... so how do I get SFR?
                    
                if (galaxyID == w3):
                    sfr = 3.77e-3 * radiusMean
                    binsMean[i] = sfr

            for i in range(len(binsSum)):
                #radius = (i + 1) * distArray.max() / 50
                radius = (i + 1) * galaxyRadius / 50
                radiusInds = np.where(distArray < radius)
                imVals = data[radiusInds]
                radiusSum = np.sum(imVals)
                
                if (galaxyID == w1):
                    ################
                    stellarMass = 3.3e2 * radiusSum
                    binsSum[i] = stellarMass
                    ################
                    
                    #same as above
                    
                if (galaxyID == w3):
                    sfr = 3.77e-3 * radiusSum
                    binsSum[i] = sfr
                
            plt.plot(radiusArray, binsMean,'b.') # create plot with radiusArray as x axis and binsMean as y axis -- small blue points are plotted
            plt.xlabel('Radius (arcseconds)') # label x axis
            plt.ylabel('Mean $\Sigma_{SFR}$ (per $kiloparsec^2$)') # label y axis
            plt.title(galaxyName + ' Mean $\Sigma_{SFR}$ Within Certain Radii') # title the graph
            if galaxyType == spiral:
                if (galaxyID == w1):
                    plt.savefig('saved avg sfr plots/w1/spiral-' + galaxyName + '-avg.png') # save WISE 1 spiral galaxy average sfr plots in their corresponding folder
                if (galaxyID == w3):
                    plt.savefig('saved avg sfr plots/w3/spiral-' + galaxyName + '-avg.png') # save WISE 3 spiral avg sfr plots
            if galaxyType == elliptical:
                if (galaxyID == w1):
                    plt.savefig('saved avg sfr plots/w1/elliptical-' + galaxyName + '-avg.png') # save WISE 1 elliptical avg sfr plots
                if (galaxyID == w3):
                    plt.savefig('saved avg sfr plots/w3/elliptical-' + galaxyName + '-avg.png') # save WISE 3 elliptical avg sfr plots
            plt.close() # close the plot to prevent overlap/errors in the program

            plt.plot(radiusArray, binsSum,'r.') # create plot with radiusArray as x axis and binsSum as y axis -- small red points are plotted
            plt.xlabel('Radius (arcseconds)')
            plt.ylabel('Sum of $\Sigma_{SFR}$ (per $kiloparsec^2$)')
            plt.title(galaxyName + ' Sum of $\Sigma_{SFR}$ Within Certain Radii')
            if galaxyType == spiral:
                if (galaxyID == w1):
                    plt.savefig('saved sum of sfr plots/w1/spiral-' + galaxyName + '-sum.png')
                if (galaxyID == w3):
                    plt.savefig('saved sum of sfr plots/w3/spiral-' + galaxyName + '-sum.png')
            if galaxyType == elliptical:
                if (galaxyID == w1):
                    plt.savefig('saved sum of sfr plots/w1/elliptical-' + galaxyName + '-sum.png')
                if (galaxyID == w3):
                    plt.savefig('saved sum of sfr plots/w3/elliptical-' + galaxyName + '-sum.png')
            plt.close()

            median = np.median(data)
            deviations = abs(data - median)
            mad = np.median(deviations)

            plt.imshow(data,vmin = median - 3*mad, vmax = median + 30*mad,origin='lower',cmap='afmhot')
            plt.title(galaxyName)
            if galaxyType == spiral:
                if (galaxyID == w1):
                    plt.savefig('saved galaxy images/w1/spiral-' + galaxyName + '.png')
                if (galaxyID == w3):
                    plt.savefig('saved galaxy images/w3/spiral-' + galaxyName + '.png')
            if galaxyType == elliptical:
                if (galaxyID == w1):
                    plt.savefig('saved galaxy images/w1/elliptical-' + galaxyName + '.png')
                if (galaxyID == w3):
                    plt.savefig('saved galaxy images/w3/elliptical-' + galaxyName + '.png')
            plt.close()

            if galaxyType == spiral:
                if (galaxyID == w1):
                    np.savetxt('saved raw data/w1/spiral/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
                if (galaxyID == w3):
                    np.savetxt('saved raw data/w3/spiral/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
            if galaxyType == elliptical:
                if (galaxyID == w1):
                    np.savetxt('saved raw data/w1/elliptical/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
                if (galaxyID == w3):
                    np.savetxt('saved raw data/w3/elliptical/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
            # save binsMean array as a .csv file, with delimiter ',', in corresponding folder
            
            if galaxyType == spiral:
                if (galaxyID == w1):
                    np.savetxt('saved raw data/w1/spiral/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')    
                if (galaxyID == w3):
                    np.savetxt('saved raw data/w3/spiral/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')
            if galaxyType == elliptical:
                if (galaxyID == w1):
                    np.savetxt('saved raw data/w1/elliptical/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')
                if (galaxyID == w3):
                    np.savetxt('saved raw data/w3/elliptical/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')
            # save binsSum array as a .csv file, with delimiter ',', in corresponding folder
            

# GALEX Data Analysis

## DONT USE DATA IT IS ALL MESSSED UP
### Problems:
- Some galaxy files are messed up and wont return data/pictures that makes sense
    - M 74, NGC 3184, NGC 3938 (fuv only)
- FIX NUMBERS TO MATCH FUV/NUV NUMBERS

In [5]:
fuvGalaxiesList = [fuv + sub for sub in galaxiesList_fits]
nuvGalaxiesList = [nuv + sub for sub in galaxiesList_fits]

galexGalaxiesList = fuvGalaxiesList + nuvGalaxiesList

galaxiesListForData = [prefix + sub for sub in galexGalaxiesList]

In [6]:
for loop in range(len(galexGalaxiesList)):
    end = len(galexGalaxiesList[loop]) - 5
    galaxyName = galexGalaxiesList[loop][4:end]
    
    for check in range(13):
        if galaxyName == galaxyInfo[check][0]:
            ########################################################
            galaxyRadius =  float(galaxyInfo[check][2]) / 3600
            ########################################################
            
            # units are off in some way still
            
            galaxyType =  galaxyInfo[check][3]
            galaxyID = galexGalaxiesList[loop][0:4]
            
            data = fits.getdata(galaxiesListForData[loop])
            header = fits.getheader(galaxiesListForData[loop])
            w = WCS(header)
            rows,cols = np.indices(data.shape)
            ra,dec = w.all_pix2world(cols,rows,1)
            ra_ctr = header['CRVAL1']
            dec_ctr = header['CRVAL2']

            distArray = np.sqrt((ra_ctr - ra)**2 + (dec_ctr - dec)**2)
            radiusArray = np.zeros(50)
            
            for i in range (len(radiusArray)):
                #radius = (i + 1) * distArray.max() / 50
                radius = (i + 1) * galaxyRadius / 50
                radiusArray[i] = radius
                
            binsMean = np.zeros(50)
            binsSum = np.zeros(50)

            for i in range(len(binsMean)):
                #radius = (i + 1) * distArray.max() / 50
                radius = (i + 1) * galaxyRadius / 50
                radiusInds = np.where(distArray < radius)
                imVals = data[radiusInds]
                radiusMean = np.mean(imVals)
                            
                if (galaxyID == fuv):
                    sfr = 1.04e-1 * radiusMean
                    binsMean[i] = sfr
                if (galaxyID == nuv):
                    sfr = 1.05e-1 * radiusMean
                    binsMean[i] = sfr

            for i in range(len(binsSum)):
                #radius = (i + 1) * distArray.max() / 50
                radius = (i + 1) * galaxyRadius / 50
                radiusInds = np.where(distArray < radius)
                imVals = data[radiusInds]
                radiusSum = np.sum(imVals)
                
                if (galaxyID == fuv):
                    sfr = 1.04e-1 * radiusSum
                    binsSum[i] = sfr
                if (galaxyID == nuv):
                    sfr = 1.05e-1 * radiusSum
                    binsSum[i] = sfr
                
            plt.plot(radiusArray, binsMean,'b.')
            plt.xlabel('Radius (arcseconds)')
            plt.ylabel('Mean $\Sigma_{SFR}$ (per $kiloparsec^2$)')
            plt.title(galaxyName + ' Mean $\Sigma_{SFR}$ Within Certain Radii')
            if galaxyType == spiral:
                if (galaxyID == nuv):
                    plt.savefig('saved avg sfr plots/nuv/spiral-' + galaxyName + '-avg.png')
                if (galaxyID == fuv):
                    plt.savefig('saved avg sfr plots/fuv/spiral-' + galaxyName + '-avg.png')
            if galaxyType == elliptical:
                if (galaxyID == nuv):
                    plt.savefig('saved avg sfr plots/nuv/elliptical-' + galaxyName + '-avg.png')
                if (galaxyID == fuv):
                    plt.savefig('saved avg sfr plots/fuv/elliptical-' + galaxyName + '-avg.png')
            plt.close()

            plt.plot(radiusArray, binsSum,'r.')
            plt.xlabel('Radius (arcseconds)')
            plt.ylabel('Sum of $\Sigma_{SFR}$ (per $kiloparsec^2$)')
            plt.title(galaxyName + ' Sum of $\Sigma_{SFR}$ Within Certain Radii')
            if galaxyType == spiral:
                if (galaxyID == nuv):
                    plt.savefig('saved sum of sfr plots/nuv/spiral-' + galaxyName + '-sum.png')
                if (galaxyID == fuv):
                    plt.savefig('saved sum of sfr plots/fuv/spiral-' + galaxyName + '-sum.png')
            if galaxyType == elliptical:
                if (galaxyID == nuv):
                    plt.savefig('saved sum of sfr plots/nuv/elliptical-' + galaxyName + '-sum.png')
                if (galaxyID == fuv):
                    plt.savefig('saved sum of sfr plots/fuv/elliptical-' + galaxyName + '-sum.png')
            plt.close()

            median = np.median(data)
            deviations = abs(data - median)
            mad = np.median(deviations)

            plt.imshow(data,vmin = median - 3*mad, vmax = median + 30*mad,origin='lower',cmap='magma')
            plt.title(galaxyName)
            if galaxyType == spiral:
                if (galaxyID == fuv):
                    plt.savefig('saved galaxy images/fuv/spiral-' + galaxyName + '.png')
                if (galaxyID == nuv):
                    plt.savefig('saved galaxy images/nuv/spiral-' + galaxyName + '.png')
            if galaxyType == elliptical:
                if (galaxyID == fuv):
                    plt.savefig('saved galaxy images/fuv/elliptical-' + galaxyName + '.png')
                if (galaxyID == nuv):
                    plt.savefig('saved galaxy images/nuv/elliptical-' + galaxyName + '.png')
            plt.close()

            if galaxyType == spiral:
                if (galaxyID == fuv):
                    np.savetxt('saved raw data/fuv/spiral/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
                if (galaxyID == nuv):
                    np.savetxt('saved raw data/nuv/spiral/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
            if galaxyType == elliptical:
                if (galaxyID == fuv):
                    np.savetxt('saved raw data/fuv/elliptical/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')
                if (galaxyID == nuv):
                    np.savetxt('saved raw data/nuv/elliptical/' + galaxyName + '-raw_mean.csv', binsMean, delimiter=',')

            if galaxyType == spiral:
                if (galaxyID == fuv):
                    np.savetxt('saved raw data/fuv/spiral/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')    
                if (galaxyID == nuv):
                    np.savetxt('saved raw data/nuv/spiral/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')
            if galaxyType == elliptical:
                if (galaxyID == fuv):
                    np.savetxt('saved raw data/fuv/elliptical/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')
                if (galaxyID == nuv):
                    np.savetxt('saved raw data/nuv/elliptical/' + galaxyName + '-raw_sum.csv', binsSum, delimiter=',')

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
